In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as st
import plotly.graph_objects as go
import missingno as msno
from sklearn import preprocessing
from sklearn.metrics import f1_score, confusion_matrix,accuracy_score, classification_report
from scipy import stats
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from isoweek import Week
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
%matplotlib inline
data = pd.read_csv('data.csv')

In [ ]:
# creating dates from year and week of the year of the data
data["date"] = 0
for i in data.index:

    input_year = data["Year"].iloc[i]
    input_week = int(data["Week of Year"].iloc[i])

    d = Week(input_year,input_week).monday()
    
    data["date"].iloc[i] = d

In [ ]:
#then we dont need this again
del data['Year']
del data['Week of Year']

In [ ]:
#using date as index
data = data.set_index('date')
data= data.sort_index()

In [ ]:
train = data[data.index < pd.to_datetime("2022-01-01", format='%Y-%m-%d')]
test = data[data.index > pd.to_datetime("2022-01-01", format='%Y-%m-%d')]

plt.plot(train, color = "black")
plt.plot(test, color = "red")
plt.ylabel('Orders')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.title("Train/Test split for order")
plt.show()

In [ ]:
#plot the info
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

plt.ylabel('Orders')
plt.xlabel('date')
plt.xticks(rotation=45)

plt.plot(data.index, data['Orders'], )

In [ ]:
import pmdarima
from pmdarima import auto_arima 
auto_arima(data['Orders'], seasonal=True, m=52,max_p=7, max_d=5,max_q=7, max_P=4, max_D=4,max_Q=4).summary()

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
arima_model = SARIMAX(train['Orders'], order = (1,0,0))
arima_result = arima_model.fit()
arima_result.summary()

In [ ]:
arima_pred = arima_result.predict(start = len(train), end = len(data)-1, typ="linear").rename("ARIMA Predictions")
arima_pred

In [ ]:
arima_pred.index = test.index
test['Orders'].plot(figsize = (16,5), legend=True)
arima_pred.plot(legend = True);